In [ ]:
import infrared as ir
from infrared import Model, Optimizer, BoltzmannSampler, def_function_class, def_constraint_class, rna

#####
## instance
tree = (((0,1),((2,3),4)),5)
print("Tree",tree)
sequences = ["AACG","ACAC","CCGG","CCAG","CCGA","UCGA"]
print("Leaves", sequences)
n = len(sequences[0])

#####
# function types
def_function_class( 'HammingDistance', lambda i,j: [i,j], lambda x,y: 1 if x!=y else 0 )
def_function_class( 'HammingDistanceLeave', lambda i,y: [i], lambda x,y: 1 if x!=y else 0 )

## char <-> integer encoding of nucleotides
ch2idx = lambda c: "ACGU".index(c)
idx2ch = lambda x: "ACGU"[x]

#####
## mode creation
def create_model( i ):
    """Create model for sequence position i"""
    model = Model()
    
    def cm(parent,tree):
        if type(tree) == int:
            model.add_functions( HammingDistanceLeave( parent, ch2idx(sequences[tree][i]) ), 'score' )
        else:
            newvar = model.num_variables
            model.add_variables( 1, (0,3), 'X' )
            if parent is not None:
                model.add_functions( HammingDistance( parent, newvar ), 'score' )
            l,r = tree
            cm( newvar, l )
            cm( newvar, r )
    
    cm( None, tree )
    
    return model


## combine single assignmnents to each inner node
def combine_assignments(ass):
    """
    @returns the sequences at the inner nodes
    """
    ass = [ x.values() for x in ass]
    m = len(ass[0])
    solution = ["".join( idx2ch(posass[i]) for posass in ass ) for i in range(m) ]

    return solution
    

#####
## Construct models, optimizer, sampler

best_ass = list()
best_scores = list()

sampler = dict()

weight = -1 ## inverse temperature controlling the sampling distribution

## create n samplers (just because we can ;))
## note: keeping all these samples simultaneously might not be the smartest way to get this done;
## we do this mostly for demonstration purposes
## also for hitting specific targets; one should rather sample the positions independently
##
for k in range(n):
    model = create_model(k)
    ## set weight 'not too low' for demonstration
    model.set_feature_weight( weight, 'score')
    optimizer = Optimizer(model)
    
    opt = optimizer.optimize()
    best_ass.append( opt )
    best_scores.append( optimizer.model.eval_feature(opt,'score') )
    
    sampler[k] = BoltzmannSampler(model)
    
best = combine_assignments(best_ass)
print(f'Optimal solution: {best}, {best_scores} {sum(best_scores)}')

#####
## Sample and print output
##
print("Samples of inner nodes:")
for i in range(10):
    score = []
    solution = None
    samples = [ sampler[k].sample() for k in range(n) ]
    scores = [ sampler[k].model.eval_feature( samples[k], 'score' ) for k in range(n) ]
    solution = combine_assignments(samples)
    print(solution,scores,sum(scores))